# Writing Software in Python

## Henry Schreiner -- 7-30-2024

## Outline

The focus today: learning how to go from research code to something reusable with structure.

We are also trying not to overlap with the packaging session tomorrow! Be sure to visit that one too.

* Scripts instead of notebooks
* Writing a CLI (built-in)
* Scripts with dependencies
* Writing a CLI (using a dependency)
* Tools for packaging
* Making a reproducible environment
* Task runners

## Requirements for today

I recommend doing this on your computer, so you'll have something to take home. Let's install:

* pipx: Brew (macOS), pip, etc: <https://pipx.pypa.io/stable>
* uv: Brew (macOS), pipx, command line: <https://github.com/astral-sh/uv>
* pixi: Brew (macOS), command line: <https://pixi.sh/latest>
* Python launcher for Unix (optional): Brew, command line: <https://python-launcher.app/install>
* nox: Brew (macOS), pipx

## Scripts instead of notebooks

Research code usually starts as notebooks. A first step is often moving some/most/all of the code out.

Before moving, a few quick tools to keep in mind for notebooks:

* `nbconvert` - can convert notebooks to html, pdf, etc. and run them too
* `papermill` - can input variables to jupyter notebooks
* You can do things like import notebooks, but 99% of the time, don't!

## Scripts

Basics:

* Start scripts with `#!/usr/bin/env python3` (called shabang) and make them executable
* Protect code with import side-effects with `if __name__ == "__main__"`

Extras:

* You can import from the same directory (packaging is better, though)
    * Can disable with PYTHONSAFEPATH (3.11+)
* Run commands directly with `python -c "..."`

**Example 1**

## Writing a CLI

The standard library provides `argparse` (and two older alternatives that are now deprecated in Python 3.13)

* Create an `ArgumentParser()` (many options)
* Add positional arguments or optional arguments (many options)
* Can also add subcommands and more
* Use `.parse_args()` to parse the args

**Example 2**

## Scripts with dependencies

There's now a standard way to add dependencies to a script:

```python
# /// script
# dependencies = [ "rich" ]
# ///
```

A growing number of tools can read this information. `pipx run`, for example.

**Example 3 (combined with next)**

## Making a CLI with click

If you want a third party tool with a different approach to command line arguments, try Click.

```python
import click

@click.command()
@click.option('--count', default=1, help='Number of greetings.')
@click.option('--name', prompt='Your name', help='The person to greet.')
def hello(count, name):
    """Simple program that greets NAME for a total of COUNT times."""
    for x in range(count):
        click.echo(f"Hello {name}!")

if __name__ == '__main__':
    hello()
```

Run with `pipx run script.py` or maybe even put in shabang line.

**Example 3 (combined with previous)**

## Tools for packaging

Specialized tools vs. all in one tools.

| Packaging need | Tool |
|----------------|------|
| Installer for libraries | ... |
| Installer for applications | ... |
| Virtual environments | ... |
| Distribution builder | ... |
| Distribution installer | ... |
| Package uploader | ... |
| Environment aware task runner | ... |

| Packaging need | Tool |
|----------------|------|
| Installer for libraries | pip |
| Installer for applications | pipx |
| Virtual environments | venv / virtualenv |
| Distribution builder | build |
| Distribution installer | installer |
| Package uploader | twine |
| Environment aware task runner | nox / tox |



All in ones:

* Poetry: the original. Replaces everything except the task runner.
* PDM: like Poetry, but follows standards and also can install Python
* Hatch: All the above except locking and also can handle tasks.
* Pixi: Conda-centric (but does support PyPI), also handles locking and tasks.

Not quite ready yet, but `uv` will also be an all-in-one, replacing `rye`.

| Other needs    | Tool   |
|----------------|--------|
| Testing        | pytest |
| Formatting     | ruff / black |
| Linting        | ruff / flake8 |
| Type checking  | mypy   |

Extra useful tools:
* Python launcher for unix: `python -> py`, finds `.venv` automatically

## Aside: pipx

Pipx is "pip for eXecutables" (or applications, but that doesn't have an x in it).

* Do you need to `import` it? Use pip.
* Do you run in from the command line? Use pipx.

You can't `import` a pipx installed app! They do not interfere with each other.

## pipx install

You can install and manage apps with pipx!

```bash
pipx install <app>
pipx inject <app> <extra-dependency>
pipx list
pipx reinstall-all
```

## pipx run

```bash
pipx run <app>
```

This is my favorite pipx feature, it downloads a PyPI package in to a temp env, then runs it. If it's done this in the last week, it's reused. You have access to all PyPI anywhere you have pipx! (GHA, etc)

This doubles as a script runner with dependencies, but this was the original use.

## pipx run examples

* **build**: make SDists and wheels
* **twine**: upload SDists and wheels
* **cibuildwheel**: make redistributable wheels
* **nox/tox**: Python task runners
* **jupylite**: WebAssembly Python site builder
* **ruff**: Python code linter and formatter
* **pypi-command-line**: query PyPI

* **uproot-browser**: ROOT file browser (HEP)
* **tiptop**: fancy top-style monitor
* **rich-cli**: pretty print files
* **cookiecutter**: template packages
* **clang-format**: format C/C++/CUDA code
* **pre-commit**: general CQA tool
* **cmake**: build system generator
* **meson**: another build system generator
* **ninja**: build system

## Setting up an environment (classic method)

### Make env

```bash
python3 -m venv .venv
```

For your main project environment, use the name `.venv`, many tools like this (including VSCode).

### Activate env

```bash
source .venv/bin/activate
```

If you don't activate, you can also fully qualify all commands, like `.venv/bin/python` _most_ of the time. 

### Install packages

```bash
pip install -r requirements.txt
```

This file is simply a list of packages to install. Info on making one in a minute.

If you are making a package, you can use a `pyproject.toml` and install requirements, but we'll see that later in the packaging session!

### Reproducible environments

```bash
pip install pip-tools
pip-compile
```

This makes a `requirements.txt` from a `requirements.in`. Add `--generate-hashes` for security.

**Example 4**

## Setting up an environment (modern uv method)

A new entry in the packaging space is uv, a Rust-based tool that replaces many classic tools but currently provides a similar one-task interface. (An all-in-one interface will be added soon, too).

```bash
uv venv
uv pip compile requirements.in
uv pip install -r requirements.txt
```

Makes non-backward compatible improvements to pip interface

* Defaults to `.venv`
* Won't install to `--system` unless requested
* Won't install `--user` at all
* Won't install _anything_ by default to venvs
* Designed to target venvs instead of being installed in them (pip can do that)

## Using an all-in-one tool

I'll demo Pixi. It uses either `pyproject.toml` or `pixi.toml`. It strongly favors conda, but can handle PyPI too. We are covering "projects" (not importable packages), so I'll use `pixi.toml`.

```bash
# New project
pixi init
pixi add click
```

Notice this installs and locks when you `add`!

```bash
# Getting existing project (locked)
pixi install
```

**Example 5**

## Task runners

Often you have various tasks that need to be run. Task runners let you define tasks along with the environment the task runs in. Nox and Tox are specific task runners, and both Hatch and Pixi also support defining tasks.

I'll use nox, as it's very flexible with Python syntax.

Running nox:

* `nox -l`: See all sessions
* `nox -s <session>`: Run a session

Making a noxfile:

```python
import nox

@nox.session
def format(session):
    session.install("ruff")
    session.run("ruff", "format", ".")
```